Baseline одного из решений

In [16]:
# импортируем библиотеки
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as m
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing


In [17]:
df = pd.read_csv('train_geo.csv')
df.head(10)

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,OFFSHORE,OIL,DEVELOPING,INVERSION/WRENCH,3520,NEOGENE,SANDSTONE,2460.0,220.0,20.0,45.0
1,GRAVITY/EXTENSION/EVAPORITE,OFFSHORE,OIL,MATURE PRODUCTION,SALT/PASSIVE MARGIN,9967,CRETACEOUS,LIMESTONE,427.0,160.0,19.0,175.0
2,GRAVITY/EXTENSION/EVAPORITE,ONSHORE,OIL,MATURE PRODUCTION,PASSIVE MARGIN,8700,CRETACEOUS,LIMESTONE,95.0,15.0,12.0,20.0
3,COMPRESSION,ONSHORE,OIL,DECLINING PRODUCTION,THRUST,5084,CRETACEOUS,SANDSTONE,328.0,300.0,13.0,600.0
4,INVERSION/COMPRESSION/EXTENSION,ONSHORE,OIL,DECLINING PRODUCTION,INVERSION/RIFT,1030,CRETACEOUS,SANDSTONE,260.0,33.0,24.0,182.0
5,COMPRESSION/EXTENSION/EVAPORITE,ONSHORE,OIL,DECLINING PRODUCTION,INTRACRATONIC,5575,CARBONIFEROUS,DOLOMITE,80.0,46.0,14.0,15.0
6,INVERSION/COMPRESSION/EXTENSION,ONSHORE,OIL,DEVELOPING,INVERSION/RIFT,5216,PROTEROZOIC,SANDSTONE,200.0,25.0,10.0,209.0
7,INVERSION/COMPRESSION/EXTENSION,OFFSHORE,OIL,DEVELOPING,RIFT,8100,CRETACEOUS,SANDSTONE,115.0,40.0,15.0,30.0
8,COMPRESSION,ONSHORE,OIL,DECLINING PRODUCTION,INTRACRATONIC,1915,CARBONIFEROUS,LIMESTONE,330.0,20.0,10.0,35.0
9,COMPRESSION,ONSHORE,OIL,DECLINING PRODUCTION,FORELAND,4150,PERMIAN,DOLOMITE,225.0,200.0,17.5,62.0


In [18]:
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [19]:
X_test = pd.read_csv('test_geo.csv')
X_test.head(10)

,Tectonic regime,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,COMPRESSION,GAS,MATURE PRODUCTION,FORELAND,2275,PERMIAN,SANDSTONE,325.0,30.0,13.0,0.04
1,INVERSION/COMPRESSION/EXTENSION/EROSION,OIL,ABANDONED,FORELAND,500,PERMIAN,SANDSTONE,40.0,18.0,28.0,240.00
2,INVERSION/COMPRESSION/EXTENSION/EVAPORITE,GAS,DECLINING PRODUCTION,INVERSION/RIFT,5548,JURASSIC,DOLOMITE,200.0,20.0,13.0,7.30
3,COMPRESSION/EVAPORITE/EXTENSION/LINKED,OIL,DEVELOPING,THRUST,10100,NEOGENE,SANDSTONE,8200.0,260.0,18.0,100.00
4,COMPRESSION/EROSION,GAS-CONDENSATE,MATURE PRODUCTION,FORELAND,8750,CARBONIFEROUS,DOLOMITE,140.0,70.0,12.0,125.00
5,EXTENSION,GAS-CONDENSATE,DECLINING PRODUCTION,RIFT,7546,CRETACEOUS,SANDSTONE,361.0,148.0,20.0,225.00
6,COMPRESSION,OIL,DECLINING PRODUCTION,INTRACRATONIC,9125,CARBONIFEROUS,DOLOMITE,105.0,40.0,16.0,25.00
7,COMPRESSION,OIL,SECOND PLATEAU PRODUTION,INTRACRATONIC,4300,CARBONIFEROUS,DOLOMITE,82.0,33.0,15.0,10.00
8,COMPRESSION,GAS,DECLINING PRODUCTION,INTRACRATONIC,1305,CRETACEOUS,SILTSTONE,30.0,10.7,20.0,2.00
9,EXTENSION,OIL,MATURE PRODUCTION,RIFT,3898,NEOGENE,SANDSTONE,2615.0,66.0,31.0,1475.00


In [20]:
y_test = pd.read_csv('answer.csv', header=None)
y_test.columns = ['Onshore/Offshore']

In [21]:
label_list = []
for column in ['Structural setting','Period', 'Lithology','Tectonic regime', 'Hydrocarbon type','Reservoir status']:
    le = preprocessing.LabelEncoder()
    df[column] = le.fit_transform(df[column].values)
    X_test[column] = le.fit_transform(X_test[column].values)
    
    

In [22]:
le = preprocessing.LabelEncoder()
df['Onshore/Offshore'] = le.fit_transform(df['Onshore/Offshore'].values)
y_test['Onshore/Offshore'] = le.fit_transform(y_test['Onshore/Offshore'].values)

In [23]:
df.head(10)

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,51,0,4,4,19,3520,12,10,2460.0,220.0,20.0,45.0
1,33,0,4,5,28,9967,6,8,427.0,160.0,19.0,175.0
2,33,1,4,5,20,8700,6,8,95.0,15.0,12.0,20.0
3,0,1,4,2,37,5084,6,10,328.0,300.0,13.0,600.0
4,41,1,4,2,17,1030,6,10,260.0,33.0,24.0,182.0
5,10,1,4,2,12,5575,3,6,80.0,46.0,14.0,15.0
6,41,1,4,4,17,5216,18,10,200.0,25.0,10.0,209.0
7,41,0,4,4,21,8100,6,10,115.0,40.0,15.0,30.0
8,0,1,4,2,12,1915,3,8,330.0,20.0,10.0,35.0
9,0,1,4,2,7,4150,17,6,225.0,200.0,17.5,62.0


In [24]:
X_train = df[['Structural setting','Period', 'Lithology','Tectonic regime', 'Hydrocarbon type','Reservoir status','Depth','Gross',	'Netpay',	'Porosity',	'Permeability']]
y_train = df['Onshore/Offshore']

In [25]:
model = GradientBoostingClassifier(random_state=0)
model.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [26]:
accuracy_score(y_test, model.predict(X_test))

0.6992481203007519

In [27]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=1,
                           depth=2,
                           loss_function='MultiClass')

In [28]:
model.fit(X_train,y_train)

 0.0126155	total: 3.2s	remaining: 2s
615:	learn: 0.0126142	total: 3.21s	remaining: 2s
616:	learn: 0.0126102	total: 3.21s	remaining: 1.99s
617:	learn: 0.0126033	total: 3.22s	remaining: 1.99s
618:	learn: 0.0125995	total: 3.22s	remaining: 1.98s
619:	learn: 0.0125925	total: 3.23s	remaining: 1.98s
620:	learn: 0.0125384	total: 3.24s	remaining: 1.98s
621:	learn: 0.0125337	total: 3.24s	remaining: 1.97s
622:	learn: 0.0125299	total: 3.25s	remaining: 1.97s
623:	learn: 0.0125074	total: 3.25s	remaining: 1.96s
624:	learn: 0.0124940	total: 3.26s	remaining: 1.96s
625:	learn: 0.0124847	total: 3.27s	remaining: 1.95s
626:	learn: 0.0124310	total: 3.27s	remaining: 1.95s
627:	learn: 0.0124240	total: 3.28s	remaining: 1.94s
628:	learn: 0.0124162	total: 3.29s	remaining: 1.94s
629:	learn: 0.0124080	total: 3.29s	remaining: 1.93s
630:	learn: 0.0123341	total: 3.3s	remaining: 1.93s
631:	learn: 0.0122985	total: 3.3s	remaining: 1.92s
632:	learn: 0.0122912	total: 3.31s	remaining: 1.92s
633:	learn: 0.0122787	total: 3.3

In [29]:
accuracy_score(y_test, model.predict(X_test))

0.7593984962406015